In [25]:
%load_ext autoreload
%autoreload 2
import aiohttp
import asyncio
import requests
import re
import time
from io import StringIO
import pandas as pd
import os
os.environ['PROXY_URL'] = ''
import json
import sys
import os
from dbase.DataAPI.ThetaData import list_contracts
from trade.helpers.pools import runProcesses


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
list_contracts?

Signature: list_contracts(symbol, start_date, print_url=False, proxy=None)
Docstring: <no docstring>
File:      ~/cloned_repos/FinanceDatabase/dbase/DataAPI/ThetaData.py
Type:      function

In [22]:
# list_contracts?
start_date = pd.to_datetime("2024-01-31").strftime('%Y%m%d')
symbol = "SPY"
symbols = ["AAPL", "TSLA", "SPY", "QQQ", "IWM", "AMZN", "NFLX", "GOOGL", "MSFT", "META", "NVDA", "HD", "VALE", "TSM", "XLY", "XLK", "XLP", "MSTR", "RIOT"]
request_list = []
thetaUrl = "http://127.0.0.1:25510/v2/list/contracts/option/trade"
for symbol in symbols:
    queryparam =  {"start_date": start_date ,"root": symbol,  "use_csv": "true"}
    request_string = f"{thetaUrl}?{'&'.join([f'{key}={value}' for key, value in queryparam.items()])}" 
    request_list.append(request_string)
request_list

['http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=AAPL&use_csv=true',
 'http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=TSLA&use_csv=true',
 'http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=SPY&use_csv=true',
 'http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=QQQ&use_csv=true',
 'http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=IWM&use_csv=true',
 'http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=AMZN&use_csv=true',
 'http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=NFLX&use_csv=true',
 'http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=GOOGL&use_csv=true',
 'http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=MSFT&use_csv=true',
 'http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=META&use_csv

In [33]:
contracts = []
for symbol in  symbols:
    contracts.append(list_contracts(symbol, start_date, print_url=True))

http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=AAPL&use_csv=true
http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=TSLA&use_csv=true
http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=SPY&use_csv=true
http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=QQQ&use_csv=true
http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=IWM&use_csv=true
http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=AMZN&use_csv=true
http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=NFLX&use_csv=true
http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=GOOGL&use_csv=true
http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=MSFT&use_csv=true
http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=META&use_csv=true
http://127.0.0.1:25510/v2/list/c

# ***ASYNC FUNCTION***

In [37]:
async def fetch(session, url):
    async with session.get(url) as response:
        return await response.text()
    
async def fetch_all(urls):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url) for url in urls]
        results = await asyncio.gather(*tasks)
        results = [pd.read_csv(StringIO(result)) for result in results]
        return results
    
await fetch_all(request_list)

[     root  expiration  strike right
 0    AAPL    20240223  225000     C
 1    AAPL    20240419  225000     C
 2    AAPL    20250321  220000     C
 3    AAPL    20240517  225000     C
 4    AAPL    20240419   95000     P
 ..    ...         ...     ...   ...
 976  AAPL    20251219  210000     P
 977  AAPL    20260116  210000     C
 978  AAPL    20260116  210000     P
 979  AAPL    20251219   80000     P
 980  AAPL    20240216  222500     C
 
 [981 rows x 4 columns],
       root  expiration  strike right
 0     TSLA    20240301  290000     C
 1     TSLA    20240301  290000     P
 2     TSLA    20240301  225000     C
 3     TSLA    20240301  225000     P
 4     TSLA    20250321  350000     C
 ...    ...         ...     ...   ...
 2181  TSLA    20241115  350000     P
 2182  TSLA    20260618  210000     C
 2183  TSLA    20260618  210000     P
 2184  TSLA    20241115  285000     P
 2185  TSLA    20241115  285000     C
 
 [2186 rows x 4 columns],
      root  expiration  strike right
 0     S

# ***MULTIPROCESSING (w pathos)***

In [35]:
list(runProcesses(list_contracts, [symbols, [start_date]*len(symbols), [True] * len(symbols)], 'imap'))

http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=SPY&use_csv=true
http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=TSLA&use_csv=true
http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=QQQ&use_csv=true
http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=AMZN&use_csv=true
http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=NFLX&use_csv=true
http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=IWM&use_csv=true
http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=GOOGL&use_csv=true
http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=AAPL&use_csv=true
http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=MSFT&use_csv=true
http://127.0.0.1:25510/v2/list/contracts/option/trade?start_date=20240131&root=META&use_csv=true
http://127.0.0.1:25510/v2/list/c

[     root  expiration  strike right
 0    AAPL    20240223   225.0     C
 1    AAPL    20240419   225.0     C
 2    AAPL    20250321   220.0     C
 3    AAPL    20240517   225.0     C
 4    AAPL    20240419    95.0     P
 ..    ...         ...     ...   ...
 976  AAPL    20251219   210.0     P
 977  AAPL    20260116   210.0     C
 978  AAPL    20260116   210.0     P
 979  AAPL    20251219    80.0     P
 980  AAPL    20240216   222.5     C
 
 [981 rows x 4 columns],
       root  expiration  strike right
 0     TSLA    20240301   290.0     C
 1     TSLA    20240301   290.0     P
 2     TSLA    20240301   225.0     C
 3     TSLA    20240301   225.0     P
 4     TSLA    20250321   350.0     C
 ...    ...         ...     ...   ...
 2181  TSLA    20241115   350.0     P
 2182  TSLA    20260618   210.0     C
 2183  TSLA    20260618   210.0     P
 2184  TSLA    20241115   285.0     P
 2185  TSLA    20241115   285.0     C
 
 [2186 rows x 4 columns],
      root  expiration  strike right
 0     S